# Single Cell Simulation Example with OBI-One Form Logic

This notebook demonstrates how to run a single cell simulation using the OBI-One form-based workflow, similar to the circuit simulation example.

In [ ]:
from entitysdk import Client
from obi_auth import get_token

from obi_notebook import get_projects
from obi_notebook import get_entities
import obi_one as obi

token = get_token(environment="production", auth_mode="daf")
project_context = get_projects.get_projects(token)

db_client = Client(environment="production", project_context=project_context, token_manager=token)

In [ ]:
# Optional: Download using unique ID
entity_ID = "<MEMODEL-ID>"  # <<< FILL IN UNIQUE MEModel ID HERE

if entity_ID != "<MEMODEL-ID>":
    memodel_ids = [entity_ID]
else:
# Alternative: Select from a table of entities
    memodel_ids = []
    memodel_ids = get_entities.get_entities("memodel", token, memodel_ids,
                                            project_context=project_context,
                                            multi_select=False,
                                            default_scale="small")



In [ ]:

# Sim duration
sim_duration = 3000.0

sim_conf = obi.MEModelSimulationScanConfig.empty_config()

# Info
info = obi.Info(campaign_name="MEModel Simulation", campaign_description="Simulation of MEModel circuit with predefined neuron set and constant current stimulus")
sim_conf.set(info, name="info")

# Regular Timesteps
regular_timestamps = obi.RegularTimestamps(start_time=0.0, number_of_repetitions=3, interval=sim_duration)
sim_conf.add(regular_timestamps, name='RegularTimestamps')

stimulus = obi.ConstantCurrentClampSomaticStimulus(
    timestamps=regular_timestamps.ref, duration=2000.0, amplitude=0.5
)
sim_conf.add(stimulus, name="CurrentClampInput")

# Recordings
voltage_recording = obi.SomaVoltageRecording()
sim_conf.add(voltage_recording, name='VoltageRecording')

time_window_voltage_recording = obi.TimeWindowSomaVoltageRecording(start_time=0.0, end_time=2000.0)
sim_conf.add(time_window_voltage_recording, name='TimeWindowVoltageRecording')

# Initialization
simulations_initialize = obi.MEModelSimulationScanConfig.Initialize(circuit=obi.MEModelFromID(id_str=memodel_ids[0]),
                                                        simulation_length=sim_duration)
sim_conf.set(simulations_initialize, name='initialize')

# Validated Config
validated_sim_conf = sim_conf.validated_config()

In [ ]:
grid_scan = obi.GridScanGenerationTask(form=validated_sim_conf, coordinate_directory_option="ZERO_INDEX", output_root='../../../obi-output/memodel_simulations/grid_scan')
grid_scan.multiple_value_parameters(display=True)
grid_scan.coordinate_parameters(display=True)
grid_scan.execute(db_client=db_client)
obi.run_tasks_for_generated_scan(grid_scan, db_client=db_client)